In [ ]:
import cv2
import mediapipe as mp
import math

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
def distancia(p1, p2):
    """
    Calculate the Euclidean distance between two points in 3D space.

    Args:
        p1: A point in 3D space represented as a named tuple with x, y, z coordinates.
        p2: A point in 3D space represented as a named tuple with x, y, z coordinates.
    """
    return math.sqrt((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2 + (p2.z - p1.z) ** 2)

In [ ]:
# funcion para determinar si un dedo esta levantado o no

# tip = punta
# pip = primera articulacion
# dip = segunda articulacion
# mcp = metacarpiana (base del dedo)
def dedo_levantado(landmarks, dedo_tip, dedo_dip, dedo_pip, dedo_mcp, umbral=0.1):
    # comparar la posición "y" del punto a la punta con los puntos anteriores
    return (
        landmarks[dedo_tip].y < landmarks[dedo_dip].y
        and landmarks[dedo_dip].y < landmarks[dedo_pip].y
        and landmarks[dedo_pip].y < landmarks[dedo_mcp].y
    )


In [ ]:
ruta = "./public/img/jpg/manos.jpg"

In [ ]:
# ruta = './public/img/jpg/mano-2.jpg
# configura la captura de la webcam
cap = cv2.VideoCapture(0)  # 0 para la webcam predeterminada

In [ ]:
# Inicializar MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Contador de frames sin manos detectadas
frames_sin_manos = 0
max_frames_sin_manos = 10000  # Por ejemplo, 100 frames (~3 segundos a 30 fps)


def is_dedo_levantado(landmarks, tip, dip, pip, mcp):
    return landmarks[tip].y < landmarks[pip].y


with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5,
) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("No se puede acceder a la cámara.")
            break

        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if not results.multi_hand_landmarks:
            frames_sin_manos += 1
            if frames_sin_manos >= max_frames_sin_manos:
                print(
                    "No se detectaron manos por un tiempo prolongado. Cerrando cámara."
                )
                break
        else:
            frames_sin_manos = 0  # Reinicia el contador si detecta una mano

            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(
                        color=(0, 255, 0), thickness=2, circle_radius=2
                    ),
                    mp_drawing.DrawingSpec(
                        color=(255, 0, 0), thickness=2, circle_radius=2
                    ),
                )

                landmarks = hand_landmarks.landmark

                dedos = {
                    "pulgar": (4, 2, 1, 0),
                    "indice": (8, 6, 5, 0),
                    "medio": (12, 10, 9, 0),
                    "anular": (16, 14, 13, 0),
                    "menique": (20, 18, 17, 0),
                }

                dedos_levantados = []
                for dedo, (tip, dip, pip, mcp) in dedos.items():
                    if is_dedo_levantado(landmarks, tip, dip, pip, mcp):
                        dedos_levantados.append(dedo)

                cv2.putText(
                    image,
                    f"Dedos levantados: {', '.join(dedos_levantados)}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2,
                )

        # Mostrar siempre la imagen, aunque no se detecten manos
        cv2.imshow("Manos", image)

        # Salir con la tecla 'q'
        # Salir con la tecla 'q' o si se cierra la ventana con la 'x'
        if (
            cv2.waitKey(5) & 0xFF == ord("q")
            or cv2.getWindowProperty("Manos", cv2.WND_PROP_VISIBLE) < 1
        ):
            break

cap.release()
cv2.destroyAllWindows()
